<a href="https://colab.research.google.com/github/preetamjumech/Hadoop_Practice/blob/main/Preetam_Saha_B1_Hadoop_WordCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hadoop
This Notebook has all the codes required to install Hadoop in the Colab VM and execute the a WordCount program using the streaming API <br>
The mapper.py and reducer.py programs are available in the authors G-Drive / Github and are downloaded as required<br>
<hr>


##Acknowledgements
Hadoop Installation from [Anjaly Sam's Github Repository](https://github.com/anjalysam/Hadoop) <br>

To get the concept behind map-reduce see [this notebook](https://github.com/Praxis-QR/BDSN/blob/main/Basic_WordCount_Concept.ipynb) <br>


# 1 Download, Install Hadoop

In [1]:
# The default JVM(Java Virtual Machine) available at /usr/lib/jvm/java-11-openjdk-amd64/  works for Hadoop
# But gives errors with Hive https://stackoverflow.com/questions/54037773/hive-exception-class-jdk-internal-loader-classloadersappclassloader-cannot
# Hence this JVM needs to be installed
!apt-get update > /dev/null #update os
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #-qq means dont throw errors on screen, /dev/null means throw errors on junk.I dont wanna see these errors

### Now, I have to download the hadoop on the machine on colab VM as a tar.gz file. gz means gun-zip

In [2]:
# If there is an error in this cell, it is very likely that the version of hadoop has changed
# Download the latest version of Hadoop and change the version numbers accordingly
#wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
#!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
!wget  https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
# Unzip it
# the tar command with the -x flag to extract, -z to uncompress, -v for verbose output, and -f to specify that we’re extracting from a file
!tar -xzf hadoop-3.3.2.tar.gz
#copy  hadoop file to user/local
!mv  hadoop-3.3.2/ /usr/local/

--2022-11-06 06:23:28--  https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638660563 (609M) [application/x-gzip]
Saving to: ‘hadoop-3.3.2.tar.gz’

hadoop-3.3.2.tar.gz 100%[===================>] 609.07M  25.5MB/s    in 25s     

2022-11-06 06:23:54 (23.9 MB/s) - ‘hadoop-3.3.2.tar.gz’ saved [638660563/638660563]



### so, I unzipped the gz file and moved it to a directory called user/local

In [3]:
!ls /usr/local

bin   cuda-11	 etc	_gcs_config_ops.so  include  licensing	sbin   src
cuda  cuda-11.2  games	hadoop-3.3.2	    lib      man	share  xgboost


### So, I checked whether the file is there or not, and yes, the file is there named hadoop-3.3.2

# 2 Set Environment Variables and system parameters


In [ ]:
#To find the default Java path
#!readlink -f /usr/bin/java | sed "s:bin/java::"
#!ls /usr/lib/jvm/

In [4]:
#To set java path, go to /usr/local/hadoop-3.3.0/etc/hadoop/hadoop-env.sh then
#. . . export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/ . . .
#we have used a simpler alternative route using os.environ - it works

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"   # default is changed
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
# make sure that the version number is as downloaded 
#os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.0/"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.2/"

### So, we have set up the java home and hadoop home, this is the part of installation

In [5]:
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [6]:
# Add Hadoop BIN to PATH
# Get the current_path from output of previous command
#current_path = '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin'
#current_path = '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin'
#new_path = current_path+':/usr/local/hadoop-3.3.2/bin/'
#os.environ["PATH"] = new_path
#!echo $PATH

current_path = os.getenv('PATH')
#new_path = current_path+':/usr/local/hadoop-3.3.0/bin/'
new_path = current_path+':/usr/local/hadoop-3.3.2/bin/'
os.environ["PATH"] = new_path

In [7]:
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/usr/local/hadoop-3.3.2/bin/


# so, previously, hadoop was not there in the path, but now it is there, so we are in the coorect path where the hadoop exists. 

# 3 Test Hadoop Installation

In [ ]:
#Running Hadoop - Test RUN, not doing anything at all
#!/usr/local/hadoop-3.3.0/bin/hadoop
# UNCOMMENT the following line if you want to make sure that Hadoop is alive!
!hadoop

### That means hadoop exists.this is a check whether hadoop is there or not

In [ ]:
# Testing Hadoop with PI generating sample program, should calculate value of pi = 3.14157500000000000000
# pi example
#Uncomment the following line if  you want to test Hadoop with pi example
# Final output should be : Estimated value of Pi is 3.14157500000000000000
#!hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar pi 16 100000
!hadoop jar /usr/local/hadoop-3.3.2/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.2.jar pi 16 100000

### so, hadoop program does its works and it calculates the value of pi.

# 4 Run WordCount with Hadoop
Instead of using Java for Map and Reduce methods, we use the streaming API of Hadoop and two simple python programs as mapper.py and reducer.py

This API(Application program interface) allows us to run hadoop from non-java languages because we are going to write our codes in python. 

In [10]:
# get mapper.py reducer.py from G_drive
#!gdown https://drive.google.com/uc?id=1VTzQ18cWAj6L29ncW6sABy-ITmDCcv5r
#!gdown https://drive.google.com/uc?id=1Or8Cbf9AsFMHStjMzDw3pXCd6TZ0dqxJ

#get mapper.py reducer.py from this git repository
!wget -q https://raw.githubusercontent.com/Praxis-QR/BDSN/main/mapper.py
!wget -q https://raw.githubusercontent.com/Praxis-QR/BDSN/main/reducer.py

### mapper generates a key-value pair, where the key being the word, and the value always 1. 
### and the reducer counts 

In [ ]:
# to see the codes, uncomment the following lines
#!cat mapper.py
#print("\n----------------------    see above for mapper, see below for reducer")
#!cat reducer.py

In [11]:
# python codes are made executable
!chmod u+rwx /content/mapper.py
!chmod u+rwx /content/reducer.py

### so, we have  changed mod to make them executable.

In [12]:
# get a simple txt file as data for word count
# or you can upload your own
#!gdown https://drive.google.com/uc?id=1R5W0UVH2S3JjPxerqyX4ue5y6tMt0Wkk
!wget -q https://raw.githubusercontent.com/Praxis-QR/BDSN/main/Chronotantra.txt

### so, we got the input file.

In [13]:
# locate the streaming jar file
!find / -name 'hadoop-streaming*.jar'

/usr/local/hadoop-3.3.2/share/hadoop/tools/lib/hadoop-streaming-3.3.2.jar
/usr/local/hadoop-3.3.2/share/hadoop/tools/sources/hadoop-streaming-3.3.2-sources.jar
/usr/local/hadoop-3.3.2/share/hadoop/tools/sources/hadoop-streaming-3.3.2-test-sources.jar
find: ‘/proc/42/task/42/net’: Invalid argument
find: ‘/proc/42/net’: Invalid argument


### so, we found the path of streaming API

In [14]:
# remove output directories
!rm -r wc_out
!rm -r wc2_out

rm: cannot remove 'wc_out': No such file or directory
rm: cannot remove 'wc2_out': No such file or directory


# hadoop and spark never overwrite directories. So, it should not exist. If it exists, hadoop will throw a huge error. 

### so, we are ready for our hadoop program, it needs four things,1. input file, 2. output directory, 3. mapper 4. reducer

In [ ]:
# execute the streaming jar with proper parameters
# four parameters are input file, output directory, the mapper progra, the reducer program
#
#!hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input /content/hobbit.txt -output /content/wc_out -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'
#!hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input /content/Chronotantra.txt -output /content/wc_out -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'
#!hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input /content/Chronotantra.txt -output /content/wc_out  -mapper 'python mapper.py'  -reducer 'python reducer.py'
!hadoop jar /usr/local/hadoop-3.3.2/share/hadoop/tools/lib/hadoop-streaming-3.3.2.jar -input /content/Chronotantra.txt -output /content/wc_out  -mapper 'python mapper.py'  -reducer 'python reducer.py'

###, so, the hadoop worked succesfully but in a single machine cluster, if it is a multi- machine clusters, the code will remain same but the installation will be more complex. 

In [16]:
# check output directory
!ls wc_out

part-00000  _SUCCESS


### hadoop generates two files, one is success file and another one is the actual output file. if success file is not there, there is failure somewhere.

In [17]:
# see actual output
#!tail wc_out/part-00000
!head wc_out/part-00000

1	8
10	2
100	2
1000	1
105	1
108	2
109	1
11	1
110	2
113	1


### this is the head of our output file

### Sorting the output

In [18]:
#https://www.geeksforgeeks.org/sort-command-linuxunix-examples/
!sort -nr -k 2 -t$'\t' wc_out/part-00000 > sorted.txt

In [19]:
!head -30 sorted.txt

would	346
could	247
one	198
time	156
like	145
know	144
us	134
mars	119
back	106
even	105
world	97
something	95
see	95
well	93
hermit	93
two	87
people	86
course	84
around	84
way	82
first	80
really	79
new	76
little	74
long	73
still	71
information	70
ai	67
good	63
earth	60


In [20]:
!tail -30 sorted.txt

2150	1
214	1
206	1
205	1
2019	1
2018	1
2007	1
20062007	1
2000	1
1999	1
1970s	1
1956	1
187	1
186	1
17866	1
156	1
155	1
15	1
150	1
1493	1
133	1
132	1
12th	1
12700	1
115	1
113	1
11	1
109	1
105	1
1000	1


In [21]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2022-11-06 13:00:04.247851+05:30
